In [1]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
import numpy as np
import warnings
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
warnings.filterwarnings("ignore")

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

In [3]:
# https://drive.google.com/file/d/1dXl0DeoYG1mkXya6mDpaY0NhbcbI17Nu/view?usp=sharing
! gdown --id 1dXl0DeoYG1mkXya6mDpaY0NhbcbI17Nu

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1dXl0DeoYG1mkXya6mDpaY0NhbcbI17Nu
To: /content/diabetes.data.csv
100% 18.5k/18.5k [00:00<00:00, 24.2MB/s]


In [4]:
data = pd.read_csv('diabetes.data.csv', sep='\t')
data.head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,135


In [5]:
target = 'Y'
X = data.drop([target], axis=1)
y = data[target]

In [6]:
lambdas = np.arange(0.05, 1.1, 0.05)
lambda_errors = []

for val in lambdas:
  model = Lasso(alpha = val)
  err = np.mean(-cross_val_score(model, X, y, cv=4, scoring='neg_mean_squared_error'))
  lambda_errors.append([val, err])

lambda_min, error_min = min(lambda_errors, key = lambda x: x[1])
print('Лучший результат достигается при lambda = ', lambda_min, ', и его MSE: ', error_min, '\n')

model = Lasso(alpha = lambda_min).fit(X, y)

print('Коэффициенты регрессии:')
print("Сдвиг = {}".format(model.intercept_))
for column, coef in zip(X.columns, model.coef_):
    print("Коэффициент для {} = {}".format(column, coef))

Лучший результат достигается при lambda =  0.05 , и его MSE:  2962.3470493269633 

Коэффициенты регрессии:
Сдвиг = -326.3434426919648
Коэффициент для AGE = -0.03529164216753992
Коэффициент для SEX = -22.589248887529163
Коэффициент для BMI = 5.61560195244985
Коэффициент для BP = 1.1153417707631796
Коэффициент для S1 = -1.0123734136498295
Коэффициент для S2 = 0.6799073849274655
Коэффициент для S3 = 0.27408411677290945
Коэффициент для S4 = 6.144165838149656
Коэффициент для S5 = 66.40492251397951
Коэффициент для S6 = 0.2827464296011796


In [7]:
lambdas = np.arange(0.05, 1.1, 0.05)
lambda_errors = []

for val in lambdas:
  model = LinearRegression()
  err = np.mean(-cross_val_score(model, X, y, cv=4, scoring='neg_mean_squared_error'))
  lambda_errors.append([val, err])

lambda_min, error_min = min(lambda_errors, key = lambda x: x[1])
print('Лучший результат достигается при lambda = ', lambda_min, ', и его MSE: ', error_min, '\n')

model = LinearRegression().fit(X, y)

print('Коэффициенты регрессии:')
print("Сдвиг = {}".format(model.intercept_))
for column, coef in zip(X.columns, model.coef_):
    print("Коэффициент для {} = {}".format(column, coef))

Лучший результат достигается при lambda =  0.05 , и его MSE:  2961.7095123449685 

Коэффициенты регрессии:
Сдвиг = -334.5671385187876
Коэффициент для AGE = -0.0363612242236198
Коэффициент для SEX = -22.859648090498375
Коэффициент для BMI = 5.6029620919237075
Коэффициент для BP = 1.1168079933181891
Коэффициент для S1 = -1.0899963340632173
Коэффициент для S2 = 0.7464504555142053
Коэффициент для S3 = 0.37200471508913036
Коэффициент для S4 = 6.5338319359903245
Коэффициент для S5 = 68.48312496478826
Коэффициент для S6 = 0.28011698932149987


С помощью кросс-валидации подобрали коэффициент регуляризации. \
Как видно из списка коэффициентов, ни один из них нельзя убрать прямо сейчас, так как все они оказывают влияние на результат\
(хотя AGE не очень сильно)


Пробуем выбросить параметр AGE. Как видим ниже, это лишь чуть улучшило результат модели

In [8]:
X1 = X.drop(['AGE'], axis = 1)
model1 = Lasso(alpha = lambda_min).fit(X1, y)
np.mean(-cross_val_score(model1, X1, y, cv=4, scoring='neg_mean_squared_error'))

2958.9514626132486

Попробуем еще возвести какие-то признаки (с большими коэффициентами) в степень, возможно они влияют нелинейно

In [9]:
X3 = X1.copy()
X3['BMI'] **= 2
X3['BP'] **= 4
X3['S5'] **= 4
X3['S4'] **= 2


model3 = Lasso(alpha=lambda_min).fit(X3, y)
np.mean(-cross_val_score(model3, X3, y, cv=4, scoring='neg_mean_squared_error'))

2895.7029343180657

Как видим, это несколько помогло улучшить результат

Поищем корреляции в столбцах:

In [10]:
correlated = data.corr()
correlated[(correlated > 0.6) & (correlated != 1.0)]

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
AGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S1,NaN,NaN,NaN,NaN,NaN,0.896663,NaN,NaN,NaN,NaN,NaN
S2,NaN,NaN,NaN,NaN,0.896663,NaN,NaN,0.659817,NaN,NaN,NaN
S3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S4,NaN,NaN,NaN,NaN,NaN,0.659817,NaN,NaN,0.617859,NaN,NaN
S5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.617859,NaN,NaN,NaN
S6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#X4 = X3.drop(['S1','S2'], axis = 1)
X4 = X3.copy()
X4 ['S12'] = X3['S1']*X3['S2']
model4 = Lasso(alpha=lambda_min).fit(X4, y)
print(np.mean(-cross_val_score(model4, X4, y, cv=4, scoring='neg_mean_squared_error')))
X4.head()

2959.352326392191


,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,S12
0,2,1030.41,104060401.0,157,93.2,38.0,16.0,557.793677,87,14632.4
1,1,466.56,57289761.0,183,103.2,70.0,9.0,229.404565,69,18885.6
2,2,930.25,74805201.0,156,93.6,41.0,16.0,476.769838,85,14601.6
3,1,640.09,49787136.0,198,131.4,40.0,25.0,571.928856,89,26017.2
4,1,529.00,104060401.0,192,125.4,52.0,16.0,338.868832,80,24076.8


Однако например, перемножение этих признаков или дроп + перемножение не дает хорошего результата(

Последняя попытка улучшить неулучшаемое: нормализация признаков (без нормализации таргета, а то я не понял, как иначе сравнить их с предыдущими результатами)

In [12]:
scaler = preprocessing.MinMaxScaler()

cols = data.drop(['Y'], axis = 1).columns
d = scaler.fit_transform(data.drop(['Y'], axis = 1))

scaled_df = pd.DataFrame(d, columns=cols)
scaled_df.head()


,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6
0,0.666667,1.0,0.582645,0.549296,0.294118,0.256972,0.207792,0.282087,0.562217,0.439394
1,0.483333,0.0,0.148760,0.352113,0.421569,0.306773,0.623377,0.141044,0.222437,0.166667
2,0.883333,1.0,0.516529,0.436620,0.289216,0.258964,0.246753,0.282087,0.496578,0.409091
3,0.083333,0.0,0.301653,0.309859,0.495098,0.447211,0.233766,0.423131,0.572923,0.469697
4,0.516667,0.0,0.206612,0.549296,0.465686,0.417331,0.389610,0.282087,0.362385,0.333333


In [13]:
model5 = Lasso(alpha=lambda_min).fit(scaled_df, y)
print(np.mean(-cross_val_score(model5, scaled_df, y, cv=4, scoring='neg_mean_squared_error')))

2961.702054877419


И да, это как-то не улучшило результат((

# Таким образом, лучший результат MSE получился равным 2895
Это было достигнуто с выкидыванием столбца AGE (он имел маленький коэффициент в регрессии), а также возведением нескольких столбцов в степень.